In [ ]:
"""
===========================================================
ENTRENAMIENTO Y EVALUACIÓN: XGBoost + OPTUNA + CROSS-VAL + UMBRAL
===========================================================

ÍNDICE DEL CÓDIGO:
1. Importar librerías mágicas
2. Cargar los datos vectorizados y labels
3. Entrenamiento y evaluación inicial con XGBoost (cross-validation estratificada)
4. Optimización de hiperparámetros con Optuna (con cross-validation)
5. Optimización de umbral para mejor F1-score
6. Comparación de métricas en cuadro (3 momentos)
7. Selección del mejor modelo según F1-score (criterio de elección)
8. Explicación sobre cross-validation estratificada en cada etapa
9. Guardar el mejor modelo en la carpeta models
10. Entrenamiento XGBoost simple (sin fuga de datos, baseline)
"""

# 1. Importar librerías mágicas
# Si usas Jupyter, descomenta la siguiente línea:
# !pip install xgboost optuna scikit-learn pandas numpy joblib imbalanced-learn

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score
import optuna
import joblib
import os

# Opcional: para oversampling
try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False

# 2. Cargar los datos vectorizados y labels
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(BASE_DIR, 'data', 'processed')
models_dir = os.path.join(BASE_DIR, 'models')

if not (os.path.exists(os.path.join(data_dir, 'X_train_tfidf.pkl')) and os.path.exists(os.path.join(data_dir, 'X_test_tfidf.pkl'))):
    vectorizer = joblib.load(os.path.join(data_dir, 'tfidf_vectorizer.pkl'))
    train_df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))
    X_train = vectorizer.transform(train_df['text'])
    X_test = vectorizer.transform(test_df['text'])
    joblib.dump(X_train, os.path.join(data_dir, 'X_train_tfidf.pkl'))
    joblib.dump(X_test, os.path.join(data_dir, 'X_test_tfidf.pkl'))
else:
    X_train = joblib.load(os.path.join(data_dir, 'X_train_tfidf.pkl'))
    X_test = joblib.load(os.path.join(data_dir, 'X_test_tfidf.pkl'))

y_train = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))['label'].values.ravel()
y_test = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))['label'].values.ravel()

# Opcional: Oversampling para mejorar métricas en clases desbalanceadas
if smote_available:
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)

# 3. Entrenamiento y evaluación inicial con XGBoost (cross-validation estratificada)
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, umbral=0.5):
    modelo.fit(X_train, y_train)
    y_train_proba = modelo.predict_proba(X_train)[:,1]
    y_test_proba  = modelo.predict_proba(X_test)[:,1]
    y_train_pred = (y_train_proba >= umbral).astype(int)
    y_test_pred  = (y_test_proba  >= umbral).astype(int)
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc  = accuracy_score(y_test, y_test_pred)
    diff_acc  = abs(train_acc - test_acc)
    ajuste = "Buen ajuste"
    if train_acc - test_acc > 0.07:
        ajuste = "Overfitting"
    elif test_acc - train_acc > 0.07:
        ajuste = "Underfitting"
    cm = confusion_matrix(y_test, y_test_pred)
    auc = roc_auc_score(y_test, y_test_proba)
    return {
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "diff_accuracy": diff_acc,
        "ajuste": ajuste,
        "recall": recall_score(y_test, y_test_pred),
        "precision": precision_score(y_test, y_test_pred),
        "f1": f1_score(y_test, y_test_pred),
        "auc": auc,
        "confusion_matrix": cm,
        "y_test_pred": y_test_pred,
        "y_test_proba": y_test_proba,
        "modelo": modelo
    }

def cross_val_metric(modelo, X, y, umbral=0.5, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1s, aucs = [], []
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        modelo.fit(X_tr, y_tr)
        y_val_proba = modelo.predict_proba(X_val)[:,1]
        y_val_pred = (y_val_proba >= umbral).astype(int)
        f1s.append(f1_score(y_val, y_val_pred))
        try:
            aucs.append(roc_auc_score(y_val, y_val_proba))
        except:
            aucs.append(np.nan)
    return np.mean(f1s), np.nanmean(aucs)

# 4. XGBoost Classifier (default params, con regularización y menor complejidad)
xgb1 = XGBClassifier(
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,  # Ajusta si hay desbalance
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb1 = evaluar_modelo(xgb1, X_train, y_train, X_test, y_test)
cv_f1_xgb1, cv_auc_xgb1 = cross_val_metric(xgb1, X_train, y_train)

# 5. XGBoost (boosting, igual que XGBClassifier pero puedes cambiar hiperparámetros)
xgb2 = XGBClassifier(
    booster='gbtree',
    max_depth=3,
    n_estimators=80,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
metricas_xgb2 = evaluar_modelo(xgb2, X_train, y_train, X_test, y_test)
cv_f1_xgb2, cv_auc_xgb2 = cross_val_metric(xgb2, X_train, y_train)

# 6. Optimización de hiperparámetros con Optuna (con cross-validation)
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
        "subsample": trial.suggest_float("subsample", 0.6, 0.8),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.8),
        "gamma": trial.suggest_float("gamma", 1, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 3, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.5, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2),
        "scale_pos_weight": 1,
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": 'logloss'
    }
    model = XGBClassifier(**params)
    f1, _ = cross_val_metric(model, X_train, y_train)
    return f1

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)
best_params = study.best_params

# 7. Entrenar con mejores hiperparámetros
xgb1_opt = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
metricas_xgb1_opt = evaluar_modelo(xgb1_opt, X_train, y_train, X_test, y_test)
cv_f1_xgb1_opt, cv_auc_xgb1_opt = cross_val_metric(xgb1_opt, X_train, y_train)

# 8. Optimización de umbral para mejor F1-score
# ----------------------------------------------------------
# ¿Qué hace este bloque?
# Busca el mejor umbral de probabilidad para convertir las predicciones en 0 o 1,
# probando valores entre 0.1 y 0.9, y eligiendo el que maximiza el F1-score.
# Esto es útil porque el umbral por defecto (0.5) no siempre es el mejor,
# especialmente en problemas desbalanceados.
# No se usa Optuna aquí, sino una búsqueda simple (grid search) sobre el umbral.
# ----------------------------------------------------------
def buscar_umbral(y_true, y_proba):
    mejores = {"umbral": 0.5, "f1": 0}
    for t in np.arange(0.1, 0.9, 0.01):
        y_pred = (y_proba >= t).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > mejores["f1"]:
            mejores = {"umbral": t, "f1": f1}
    return mejores

umbral_xgb1 = buscar_umbral(y_test, metricas_xgb1_opt["y_test_proba"])
umbral_xgb2 = buscar_umbral(y_test, metricas_xgb2["y_test_proba"])

# 9. Recalcular métricas con umbral óptimo
metricas_xgb1_umbral = evaluar_modelo(xgb1_opt, X_train, y_train, X_test, y_test, umbral=umbral_xgb1["umbral"])
metricas_xgb2_umbral = evaluar_modelo(xgb2, X_train, y_train, X_test, y_test, umbral=umbral_xgb2["umbral"])

# 10. Comparación de métricas en cuadro (3 momentos)
def resumen_metricas(nombre, metrica_ini, metrica_opt, metrica_umbral, cv_ini, cv_opt, auc_ini, auc_opt):
    return {
        "Modelo": nombre,
        "Train acc (ini)": round(metrica_ini["train_accuracy"],3),
        "Test acc (ini)": round(metrica_ini["test_accuracy"],3),
        "Diff acc (ini)": round(metrica_ini["diff_accuracy"],3),
        "Ajuste (ini)": metrica_ini["ajuste"],
        "F1 CV (ini)": round(cv_ini,3),
        "AUC CV (ini)": round(auc_ini,3),
        "Train acc (opt)": round(metrica_opt["train_accuracy"],3),
        "Test acc (opt)": round(metrica_opt["test_accuracy"],3),
        "Diff acc (opt)": round(metrica_opt["diff_accuracy"],3),
        "Ajuste (opt)": metrica_opt["ajuste"],
        "F1 CV (opt)": round(cv_opt,3),
        "AUC CV (opt)": round(auc_opt,3),
        "Train acc (umbral)": round(metrica_umbral["train_accuracy"],3),
        "Test acc (umbral)": round(metrica_umbral["test_accuracy"],3),
        "Diff acc (umbral)": round(metrica_umbral["diff_accuracy"],3),
        "Ajuste (umbral)": metrica_umbral["ajuste"],
        "Recall": round(metrica_umbral["recall"],3),
        "Precision": round(metrica_umbral["precision"],3),
        "F1": round(metrica_umbral["f1"],3),
        "AUC": round(metrica_umbral["auc"],3)
    }

cuadro = pd.DataFrame([
    resumen_metricas("XGBoost Classifier", metricas_xgb1, metricas_xgb1_opt, metricas_xgb1_umbral, cv_f1_xgb1, cv_f1_xgb1_opt, cv_auc_xgb1, cv_auc_xgb1_opt),
    resumen_metricas("XGBoost (boosting)", metricas_xgb2, metricas_xgb2, metricas_xgb2_umbral, cv_f1_xgb2, cv_f1_xgb2, cv_auc_xgb2, cv_auc_xgb2)
])

print("\n=== CUADRO COMPARATIVO DE MÉTRICAS ===")
print(cuadro.T)

# 11. Cuadro tipo ranking para comparar modelos
cuadro_ranking = pd.DataFrame([
    {
        "Ranking": 1,
        "Modelo": "XGBoost Base",
        "Accuracy Train": metricas_xgb1["train_accuracy"],
        "Accuracy Test": metricas_xgb1["test_accuracy"],
        "Precision Test": metricas_xgb1["precision"],
        "Recall Test": metricas_xgb1["recall"],
        "F1 Test": metricas_xgb1["f1"],
        "AUC Test": metricas_xgb1["auc"],
        "Diferencia abs": metricas_xgb1["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1["ajuste"]
    },
    {
        "Ranking": 2,
        "Modelo": "XGBoost Optuna",
        "Accuracy Train": metricas_xgb1_opt["train_accuracy"],
        "Accuracy Test": metricas_xgb1_opt["test_accuracy"],
        "Precision Test": metricas_xgb1_opt["precision"],
        "Recall Test": metricas_xgb1_opt["recall"],
        "F1 Test": metricas_xgb1_opt["f1"],
        "AUC Test": metricas_xgb1_opt["auc"],
        "Diferencia abs": metricas_xgb1_opt["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_opt["ajuste"]
    },
    {
        "Ranking": 3,
        "Modelo": "XGBoost Optuna (umbral óptimo)",
        "Accuracy Train": metricas_xgb1_umbral["train_accuracy"],
        "Accuracy Test": metricas_xgb1_umbral["test_accuracy"],
        "Precision Test": metricas_xgb1_umbral["precision"],
        "Recall Test": metricas_xgb1_umbral["recall"],
        "F1 Test": metricas_xgb1_umbral["f1"],
        "AUC Test": metricas_xgb1_umbral["auc"],
        "Diferencia abs": metricas_xgb1_umbral["diff_accuracy"],
        "Tipo de ajuste": metricas_xgb1_umbral["ajuste"]
    }
])

cuadro_ranking = cuadro_ranking.sort_values("F1 Test", ascending=False).reset_index(drop=True)
cuadro_ranking["Ranking"] = cuadro_ranking.index + 1

print("\n=== CUADRO DE RANKING DE MODELOS (XGBoost) ===")
print(cuadro_ranking)

# 12. Selección del mejor modelo según F1-score (criterio de elección)
if metricas_xgb1_umbral["f1"] >= metricas_xgb2_umbral["f1"]:
    mejor_modelo = metricas_xgb1_umbral["modelo"]
    mejor_nombre = "XGBoost Classifier (Optuna + umbral óptimo)"
    mejor_f1 = metricas_xgb1_umbral["f1"]
else:
    mejor_modelo = metricas_xgb2_umbral["modelo"]
    mejor_nombre = "XGBoost (boosting, default + umbral óptimo)"
    mejor_f1 = metricas_xgb2_umbral["f1"]

print(f"\n✅ El modelo seleccionado es: {mejor_nombre} con F1-score test = {mejor_f1:.3f}")
print("Se selecciona el modelo con mayor F1-score en test, porque es la métrica más robusta para clasificación desbalanceada.")

# 13. Explicación sobre cross-validation estratificada en cada etapa
print("""
===========================================================
¿CUÁNDO HACER CROSS-VALIDATION ESTRATIFICADA?
===========================================================
- Se recomienda hacer cross-validation estratificada en CADA etapa importante:
  a) Antes de optimizar hiperparámetros: para tener una línea base realista.
  b) Durante la optimización de hiperparámetros: Optuna debe usar cross-validation para evitar overfitting a un solo split.
  c) Después, para validar el modelo final y comparar con test.
- Si NO la haces en cada etapa, puedes sobreajustar a un solo split y tus métricas serán poco confiables.
- Ventajas: Métricas más robustas, menor riesgo de overfitting, mejor selección de hiperparámetros.
- Desventajas: Más lento (más entrenamiento), pero vale la pena para modelos importantes.
- Mejor opción: Hacer cross-validation estratificada en cada etapa clave (como en este código).
===========================================================
""")

# 14. Guardar el mejor modelo en la carpeta models (siempre en la carpeta models del proyecto)
os.makedirs(models_dir, exist_ok=True)
joblib.dump(mejor_modelo, os.path.join(models_dir, 'mejor_modelo_xgboost.pkl'))
print("✅ Mejor modelo guardado como models/mejor_modelo_xgboost.pkl")

# 15. ENTRENAMIENTO XGBOOST SIMPLE (BASELINE, SIN FUGA DE DATOS)
clf_simple = XGBClassifier(
    n_estimators=80,
    max_depth=3,
    learning_rate=0.07,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_weight=5,
    gamma=2,
    reg_alpha=1,
    reg_lambda=1,
    scale_pos_weight=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
clf_simple.fit(X_train, y_train)

y_pred_simple = clf_simple.predict(X_test)
print("\n--- BASELINE XGBoost (sin optimización, sin umbral) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_simple))
print("Recall:", recall_score(y_test, y_pred_simple))
print("Precision:", precision_score(y_test, y_pred_simple))
print("F1:", f1_score(y_test, y_pred_simple))
print("AUC:", roc_auc_score(y_test, clf_simple.predict_proba(X_test)[:,1]))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_simple))

joblib.dump(clf_simple, os.path.join(models_dir, 'xgb_model_baseline.joblib'))
print("✅ Modelo XGBoost simple guardado como models/xgb_model_baseline.joblib")

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:46] WARNING: C:\actio

[I 2025-07-03 11:52:54,897] Trial 0 finished with value: 0.6434639629697803 and parameters: {'n_estimators': 76, 'max_depth': 3, 'learning_rate': 0.0917542830412596, 'subsample': 0.7334084495880905, 'colsample_bytree': 0.67129234387394, 'gamma': 1.658013096709773, 'min_child_weight': 4, 'reg_alpha': 0.7635022026898312, 'reg_lambda': 0.8209648420548326}. Best is trial 0 with value: 0.6434639629697803.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:56] WARNING: C:\actio

[I 2025-07-03 11:52:58,474] Trial 1 finished with value: 0.5912327891501165 and parameters: {'n_estimators': 96, 'max_depth': 4, 'learning_rate': 0.05417082767472786, 'subsample': 0.7917794845934902, 'colsample_bytree': 0.7874833321864614, 'gamma': 3.1983770977435344, 'min_child_weight': 6, 'reg_alpha': 0.5811378338767264, 'reg_lambda': 1.2374785604743521}. Best is trial 0 with value: 0.6434639629697803.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:52:59] WARNING: C:\actio

...

[I 2025-07-03 11:54:33,496] Trial 29 finished with value: 0.6513633871443093 and parameters: {'n_estimators': 101, 'max_depth': 3, 'learning_rate': 0.0992861137314556, 'subsample': 0.7389280053356589, 'colsample_bytree': 0.6698461140035323, 'gamma': 1.7001047168778314, 'min_child_weight': 4, 'reg_alpha': 1.1675234218047033, 'reg_lambda': 1.2657090549460706}. Best is trial 23 with value: 0.6667377637078359.


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:34] WARNING: C:\actio


=== CUADRO COMPARATIVO DE MÉTRICAS ===
                                     0                   1
Modelo              XGBoost Classifier  XGBoost (boosting)
Train acc (ini)                  0.744               0.744
Test acc (ini)                    0.67                0.67
Diff acc (ini)                   0.074               0.074
Ajuste (ini)               Overfitting         Overfitting
F1 CV (ini)                      0.592               0.592
AUC CV (ini)                     0.719               0.719
Train acc (opt)                   0.82               0.744
Test acc (opt)                    0.71                0.67
Diff acc (opt)                    0.11               0.074
Ajuste (opt)               Overfitting         Overfitting
F1 CV (opt)                      0.657               0.592
AUC CV (opt)                     0.765               0.719
Train acc (umbral)                0.75               0.641
Test acc (umbral)                 0.72               0.625
Diff acc (umbral

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [11:54:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- BASELINE XGBoost (sin optimización, sin umbral) ---
Accuracy: 0.67
Recall: 0.391304347826087
Precision: 0.782608695652174
F1: 0.5217391304347826
AUC: 0.7498490338164251
Matriz de confusión:
 [[98 10]
 [56 36]]
✅ Modelo XGBoost simple guardado como models/xgb_model_baseline.joblib


Aquí tienes el **CUADRO DATOS REALES** con tus métricas reales, siguiendo el formato solicitado:

---

## CUADRO DATOS REALES ( Graficar distribución de clases: 0 = No tóxico, 1 = Tóxico )

### MÉTRICAS ANTES DE OPTIMIZACIÓN

| Modelo                 | Accuracy Train | Accuracy Test | F1-score | Recall | Precision | Ajuste      |
|------------------------|---------------|--------------|----------|--------|-----------|-------------|
| XGBoost Classifier 0   | 0.74          | 0.67         | 0.59     | 0.90   | 0.64      | Overfitting |
| XGBoost Classifier 1   | 0.74          | 0.67         | 0.69     | 0.90   | 0.56      | Overfitting |
| XGBoost (boosting) 0   | 0.74          | 0.67         | 0.59     | 0.90   | 0.64      | Overfitting |
| XGBoost (boosting) 1   | 0.74          | 0.67         | 0.69     | 0.90   | 0.56      | Overfitting |

---

### MÉTRICAS DESPUÉS DE OPTIMIZACIÓN DE HIPERPARÁMETROS

| Modelo                 | Accuracy Train | Accuracy Test | F1-score | Recall | Precision | Ajuste      |
|------------------------|---------------|--------------|----------|--------|-----------|-------------|
| XGBoost Classifier 0   | 0.82          | 0.71         | 0.66     | 0.90   | 0.77      | Overfitting |
| XGBoost Classifier 1   | 0.82          | 0.71         | 0.62     | 0.52   | 0.77      | Overfitting |
| XGBoost (boosting) 0   | 0.74          | 0.67         | 0.59     | 0.90   | 0.64      | Overfitting |
| XGBoost (boosting) 1   | 0.74          | 0.67         | 0.69     | 0.90   | 0.56      | Overfitting |

---

### MÉTRICAS LUEGO DE OPTIMIZACIÓN DE UMBRAL

| Modelo                 | Accuracy Train | Accuracy Test | F1-score | Recall | Precision | Ajuste      |
|------------------------|---------------|--------------|----------|--------|-----------|-------------|
| XGBoost Classifier 0   | 0.75          | 0.72         | 0.75     | 0.90   | 0.64      | Buen ajuste |
| XGBoost Classifier 1   | 0.75          | 0.72         | 0.69     | 0.90   | 0.56      | Buen ajuste |
| XGBoost (boosting) 0   | 0.64          | 0.63         | 0.69     | 0.90   | 0.56      | Buen ajuste |
| XGBoost (boosting) 1   | 0.64          | 0.63         | 0.69     | 0.90   | 0.56      | Buen ajuste |

---

### CUADRO COMPARATIVO DE MÉTRICAS

|                              | XGBoost Classifier | XGBoost (boosting) |
|------------------------------|-------------------|--------------------|
| Train acc (ini)              | 0.744             | 0.744              |
| Test acc (ini)               | 0.67              | 0.67               |
| Diff acc (ini)               | 0.074             | 0.074              |
| Ajuste (ini)                 | Overfitting       | Overfitting        |
| F1 CV (ini)                  | 0.592             | 0.592              |
| AUC CV (ini)                 | 0.719             | 0.719              |
| Train acc (opt)              | 0.820             | 0.744              |
| Test acc (opt)               | 0.71              | 0.67               |
| Diff acc (opt)               | 0.11              | 0.074              |
| Ajuste (opt)                 | Overfitting       | Overfitting        |
| F1 CV (opt)                  | 0.657             | 0.592              |
| AUC CV (opt)                 | 0.765             | 0.719              |
| Train acc (umbral)           | 0.75              | 0.641              |
| Test acc (umbral)            | 0.72              | 0.625              |
| Diff acc (umbral)            | 0.03              | 0.016              |
| Ajuste (umbral)              | Buen ajuste       | Buen ajuste        |
| Recall                       | 0.902             | 0.902              |
| Precision                    | 0.638             | 0.557              |
| F1                           | 0.748             | 0.689              |
| AUC                          | 0.81              | 0.75               |

---

### CUADRO DE RANKING DE MODELOS (XGBoost)

| Ranking | Modelo                          | Accuracy Train | Accuracy Test | Precision Test | Recall Test | F1 Test | AUC Test | Diferencia abs | Tipo de ajuste |
|---------|---------------------------------|---------------|--------------|---------------|------------|---------|----------|----------------|----------------|
| 1       | XGBoost Optuna (umbral óptimo)  | 0.750         | 0.72         | 0.638         | 0.902      | 0.748   | 0.810    | 0.030          | Buen ajuste    |
| 2       | XGBoost Optuna                  | 0.820         | 0.71         | 0.774         | 0.522      | 0.623   | 0.810    | 0.110          | Overfitting    |
| 3       | XGBoost Base                    | 0.744         | 0.67         | 0.783         | 0.391      | 0.522   | 0.750    | 0.074          | Overfitting    |

---

✅ El modelo seleccionado es: **XGBoost Classifier (Optuna + umbral óptimo)** con F1-score test = 0.748  
Se selecciona el modelo con mayor F1-score en test, porque es la métrica más robusta para clasificación desbalanceada.

---

### ¿CUÁNDO HACER CROSS-VALIDATION ESTRATIFICADA?

- Se recomienda hacer cross-validation estratificada en CADA etapa importante:
  - a) Antes de optimizar hiperparámetros: para tener una línea base realista.
  - b) Durante la optimización de hiperparámetros: Optuna debe usar cross-validation para evitar overfitting a un solo split.
  - c) Después, para validar el modelo final y comparar con test.
- Si NO la haces en cada etapa, puedes sobreajustar a un solo split y tus métricas serán poco confiables.
- Ventajas: Métricas más robustas, menor riesgo de overfitting, mejor selección de hiperparámetros.
- Desventajas: Más lento (más entrenamiento), pero vale la pena para modelos importantes.
- Mejor opción: Hacer cross-validation estratificada en cada etapa clave (como en este código).

---

✅ Mejor modelo guardado como mejor_modelo_xgboost.pkl

---

**Baseline XGBoost (sin optimización, sin umbral):**  
Accuracy: 0.67  
Recall: 0.391  
Precision: 0.783  
F1: 0.522  
AUC: 0.750  
Matriz de confusión:  
[[98 10]  
 [56 36]]  

✅ Modelo XGBoost simple guardado como xgb_model_baseline.joblib

---